##  重试解析器

虽然在某些情况下，可以仅通过查看输出来修复任何解析错误，但在其他情况下则不然。例如，输出不仅格式不正确，而且部分完整。考虑下面的例子。

In [8]:
import pprint
from langchain.output_parsers import (
    OutputFixingParser,
    PydanticOutputParser,
)
from langchain.prompts import (
    PromptTemplate,
)
# from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel,Field
from langchain_openai import ChatOpenAI, OpenAI
API_KEY="sk-9846f14a2104490b960adbf5c5b3b32e" #老师的Key
BASE_URL = "https://api.deepseek.com"
MODEL= "deepseek-chat"

chat = ChatOpenAI(
    model= MODEL,  # 默认的大模型为GPT-3.5-turbo，比较便宜
    openai_api_base= BASE_URL,
    openai_api_key= API_KEY,
    temperature=0.3,
)


In [9]:
# template = """根据用户问题，提供应采取的操作和操作输入。
# {format_instructions}
# 问题: {query}
# 答复:"""


class Action(BaseModel):
    action: str = Field(description="采取的行动")
    action_input: str = Field(description="输入到动作")


parser = PydanticOutputParser(pydantic_object=Action)

In [10]:
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"action": {"description": "采取的行动", "title": "Action", "type": "string"}, "action_input": {"description": "输入到动作", "title": "Action Input", "type": "string"}}, "required": ["action", "action_input"]}\n```'

In [11]:
prompt = PromptTemplate(
    template="回答用户的询问。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [12]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

In [15]:
print(prompt_value.text)


回答用户的询问。
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"action": {"description": "采取的行动", "title": "Action", "type": "string"}, "action_input": {"description": "输入到动作", "title": "Action Input", "type": "string"}}, "required": ["action", "action_input"]}
```
who is leo di caprios gf?



In [16]:
testLLM = prompt | chat 
testLLM.invoke({"query": "who is leo di caprios gf?"})

AIMessage(content='```json\n{\n  "action": "search",\n  "action_input": "Leo DiCaprio current girlfriend 2023"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 201, 'total_tokens': 230, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 201}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': 'ff93c647-b1e1-4b3d-832c-74f40721facd', 'finish_reason': 'stop', 'logprobs': None}, id='run-48260df5-e79d-4f32-bbb4-1d97f114fcce-0', usage_metadata={'input_tokens': 201, 'output_tokens': 29, 'total_tokens': 230, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [17]:
chain = prompt | chat | parser
parser_output = chain.invoke({"query": "who is leo di caprios gf?"})

parser_output

Action(action='search', action_input='current girlfriend of Leonardo DiCaprio')

In [18]:
bad_response = '{"action": "search"}'

In [19]:
parser.parse(bad_response)

OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [20]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=chat)

In [21]:
fix_parser.parse(bad_response)

Action(action='search', action_input='query')

In [22]:
from langchain.output_parsers import RetryOutputParser
retry_parser = RetryOutputParser.from_llm(parser=parser, llm=chat)



In [23]:
retry_parser.parse_with_prompt(bad_response, prompt_value)

Action(action='search', action_input="Who is Leonardo DiCaprio's girlfriend?")